In [11]:

import numpy as np 
import pandas as pd
from nilearn.datasets import fetch_haxby
from nilearn.input_data import NiftiMasker

from reciprocal_isomap import ReciprocalIsomap

from kmapper import KeplerMapper
from landmark_cover import LandmarkCover
from sklearn.cluster import AgglomerativeClustering


/home/poetz/anaconda3/envs/tda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

# Fetch dataset, extract time-series from ventral temporal (VT) mask
dataset = fetch_haxby(subjects=[2])
masker = NiftiMasker(
    dataset.mask_vt[0], 
    standardize=True, detrend=True, smoothing_fwhm=4.0,
    low_pass=0.09, high_pass=0.008, t_r=2.5,
    memory="nilearn_cache")
X = masker.fit_transform(dataset.func[0])

# Encode labels as integers
df = pd.read_csv(dataset.session_target[0], sep=" ")
target, labels = pd.factorize(df.labels.values)
timing = pd.DataFrame().assign(task=target, task_name=labels[target])
timing_onehot = pd.DataFrame({l:1*(target==i) for i,l in enumerate(labels)})

In [39]:
# compute distances with reciprocal isomap


# compute geodesic distances on a reciprocal neighbor graph
r_isomap = ReciprocalIsomap(n_neighbors=30,metric='correlation', neighbors_mode='connectivity')
geodesic_distances = r_isomap._reciprocal_distances(X).toarray()


In [41]:
#feed distances to KeplerMapper using Landmark Cover



# setup KeplerMapper object
mapper = KeplerMapper(verbose=1)

# run KeplerMapper using the LandmarkCover
graph = mapper.map(
    lens=geodesic_distances, X=X,
    cover=LandmarkCover(n_landmarks=50, perc_overlap=0.4, metric='precomputed'),
    clusterer=AgglomerativeClustering(n_clusters=3, linkage='single'),
    remove_duplicate_nodes=True,
)

# visualize the graph
html = mapper.visualize(graph, path_html=f'kepler.html')

KeplerMapper()
Mapping on data shaped (1452, 464) using lens shaped (1452, 1452)

Creating 50 hypercubes.
Merged 47 duplicate nodes.

Number of nodes before merger: 150; after merger: 103


Created 2850 edges and 103 nodes in 0:00:07.480731.
Wrote visualization to: kepler.html


In [42]:
## Visualize the shape graph using DyNeuSR's DyNeuGraph
from dyneusr import DyNeuGraph

dG = DyNeuGraph(G=graph, y=timing_onehot)
dG.visualize('trial_0.html')

label,group,value,row_count
rest,0,588,1452
scissors,1,108,1452
face,2,108,1452
cat,3,108,1452
shoe,4,108,1452
house,5,108,1452
scrambledpix,6,108,1452
bottle,7,108,1452
chair,8,108,1452


Already serving localhost:None 
[Force Graph] file:///home/poetz/phd_projects/CSSS22-TDA/code/trial_0.html


DyNeuGraph()